# ESP32 RTS/CTS

In [1]:
%discover
%connect esp32_cop
%rsync
%softreset

Hostname             URL                                      UID
nrf52                serial:///dev/cu.usbmodem1413401         c7:9d:75:c8:7a:14:1d:b6
esp32_cop            serial:///dev/cu.usbserial-014352DD      30:ae:a4:1a:27:28
Connected to esp32_cop @ serial:///dev/cu.usbserial-014352DD
with projects [secrets, airlift/libs/server]
UPDATE  lib/urpc/server.py

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



## Open

In [1]:
%connect esp32_cop

import machine, time, os

config = { 'rx': 16, 'tx': 17, 'rts': 18, 'cts': 19, 'baudrate': 1000000 }
uart = machine.UART(2, **config)
uart.init(bits=8, stop=1, txbuf=256, rxbuf=256, timeout=500)


Connected to esp32_cop @ serial:///dev/cu.usbserial-014352DD
with projects [boards/esp32/mcu, secrets, airlift]


* After reset: all pins "low" (really "open")
* Init uart: all pins high, except RTS
* CTS is in, RTS is out
* --> unexpected RX, CTS high???

## Loopback

In [1]:
%connect esp32_cop

import machine, time, os

config = { 'rx': 16, 'tx': 17, 'baudrate': 1000000, 'rts': 18, 'cts': 19 }
uart = machine.UART(2, **config)
uart.init(bits=8, stop=1, txbuf=256, rxbuf=256, timeout=500)

uart.write(b'abcdefghijklmnop')
print("wrote")
for i in range(5):
    iw = uart.any()
    res = uart.read(iw)
    print("[{:3}] {}".format(iw, res))
    time.sleep(0.1)

Connected to esp32_cop @ serial:///dev/cu.usbserial-014352DD
with projects [boards/esp32/mcu, secrets, airlift]
wrote
[ 16] b'abcdefghijklmnop'
[  0] b''
[  0] b''
[  0] b''
[  0] b''


In [1]:
for i in range(20):
    x = uart.write(b'abcdefghijklmnop')

Interrupted

In [1]:
for i in range(5):
    iw = uart.any()
    res = uart.read(iw)
    print("[{:3}] {}".format(iw, res))
    time.sleep(0.1)

Timeout reading from MCU, got bytearray(b''), expect b'raw REPL; CTRL-B to exit\r\n>'

* pull CTS input low to receive data
* RTS output is low

# with NRF

In [1]:
%connect esp32_cop

import machine, time, os, io

config = { 'rx': 16, 'tx': 17, 'baudrate': 1000000, 'rts': 18, 'cts': 19 }
uart = machine.UART(2, **config)
uart.init(bits=8, stop=1, txbuf=256, rxbuf=256, timeout=500)

print("empty buffer ...", uart.any())
while uart.any():
    uart.read(uart.any())
    time.sleep(0.1)

N=700
data = bytearray(N)
b = io.BytesIO()

c = 200
for i in range(N):
    if c > 120:
        c = ord('0')
    data[i] = c
    c += 1

print("writer data")
uart.write(data)

while len(b.getvalue()) < N:
    iw = uart.any()
    if iw:
        res = uart.read(iw)
        b.write(res)
        print("got [{:3}]  {}".format(iw, res))
    else:
        time.sleep(0.1)
        
if data != b.getvalue():
    print("Mismatch!")
else:
    print("Match!")

Connected to esp32_cop @ serial:///dev/cu.usbserial-014352DD
with projects [secrets, airlift/libs/server]
empty buffer ... 0
writer data
got [256]  b'0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwx0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwx0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwx0123456789:;<=>?@ABCDEFGHIJKLMNOPQRST'
got [256]  b'UVWXYZ[\\]^_`abcdefghijklmnopqrstuvwx0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwx0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwx0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwx0'
got [188]  b'123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwx0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwx0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ'
Match!


## urpc server

In [1]:
%connect esp32_cop
%rsync

Connected to esp32_cop @ serial:///dev/cu.usbserial-014352DD
with projects [secrets, airlift/libs/server]
Directories match


In [1]:
%connect esp32_cop

import uasyncio as asyncio
import msgpack, io, os

# object registry
# concrete objects on server are represented by _Proxy on client
# format: id(object) --> object
_registry = {}


def im(module):
    """Import module"""
    exec("import {}".format(module))
    return locals()[module]

def cm(obj, name, args, kwargs):
    """Call instance method"""
    return getattr(obj, name)(*args, **kwargs)
    
def gp(obj, name):
    """Get property"""
    return getattr(obj, name)
    
def sp(obj, name, value):
    """Set property"""
    return setattr(obj, name, value)

def _d(obj):
    """Remove object from registry, __del__"""
    try:
        del _registry[id(obj)]
    except KeyError:
        pass
    
def _s(obj):
    """__str__"""
    return str(obj)

def _r():
    """Registry (backdoor)"""
    return _registry
    

def _ext_handler(code, data):
    # convert ExtType to object
    if code != 1:
        print("_ext_handler({} ?, {})".format(code, data))
    return _registry.get(int.from_bytes(data, 'big'))


def _obj_handler(obj):
    # convert object to ExtType
    x = id(obj)
    if not _registry.get(x):
        _registry[x] = obj
    return msgpack.ExtType(1, x.to_bytes(4, 'big'))
    

class Server(io.IOBase):
    
    def __init__(self, uart):
        self.uart = uart
        
    # for dupterm
    def write(self, data):
        if not self.print:
            self.print = io.StringIO()
        self.print.write(data)

    # for dupterm
    def readinto(self, *args):
        return None

    def serve(self):
        # Serve one request. Blocking.
        call = msgpack.unpack(self.uart, ext_hook=_ext_handler, use_list=False)
        self.print = None
        os.dupterm(self)
        try:
            # blocks however long the user's code takes (e.g. socket.recv)
            result = globals().get(call[0])(*call[1:])
        except Exception as e:
            s = io.StringIO()
            sys.print_exception(e, s)
            result = msgpack.ExtType(2, s.getvalue().encode())
        finally:
            os.dupterm(None)
        if self.print:
            # output from print statements
            msgpack.pack(msgpack.ExtType(3, self.print.getvalue()), self.uart)
        msgpack.pack(result, self.uart, default=_obj_handler)

    
async def async_serve(uart):
    # serve requests indefinitely
    # we use a Stream object to await read(0)
    # but do the actual reading (with msgpack) from the uart itself
    server = Server(uart)
    stream = asyncio.StreamReader(uart)
    n = 0
    while True:
        try:
            # wait for request 
            # (we assume stream.read isn't clever optimizes away the read(0) ...)
            n += 1
            await stream.read(0)
            server.serve()
        except Exception as e:
            # Internal server error. Should never happen.
            try:
                error = msgpack.ExtType(2, repr("PRC Server Internal Error: {}".format(e)).encode())
                msgpack.pack(error, uart)
            except:
                pass

            
configuration = {
    # particle-argon with on-board esp32
    'argon':      { 'rx': 19, 'tx': 22, 'baudrate': 1000000 },
    'argon_flow': { 'rx': 19, 'tx': 22, 'rts': 0, 'cts': 26, 'baudrate': 1_000_000 },

    # huzzah32 as esp32 co-processor. tried on feather stm32f405.
    'huzzah32':   { 'rx': 16, 'tx': 17, 'baudrate': 1000000, 'rts': 18, 'cts': 19 },
}


def start_server(config, enable_net_server=False):

    uart = machine.UART(2, **config)
    uart.init(bits=8, stop=1, txbuf=1024, rxbuf=1024, timeout=20000)

    async def main(uart):
        print("starting urpc server")
        asyncio.create_task(async_serve(uart))
        
        if enable_net_server:
            import net_repl
            import secrets
            asyncio.create_task(net_repl.async_serve(
                port=54321, 
                password=getattr(secrets, 'net_pwd', None)))
            
        asyncio.get_event_loop().run_forever()

    asyncio.run(main(uart))

    
config = configuration['huzzah32']
start_server(config, enable_net_server=False)

Connected to esp32_cop @ serial:///dev/cu.usbserial-014352DD
with projects [secrets, airlift/libs/server]
starting urpc server
ets Jun  8 2016 00:22:57

rst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0018,len:4
load:0x3fff001c,len:5120
ho 0 tail 12 room 4
load:0x40078000,len:10680
ho 0 tail 12 room 4
load:0x40080400,len:5684
entry 0x400806bc
MicroPython v1.13-274-ga1853285f-dirty on 2021-01-03; 4MB/OTA module with ESP32
Type "help()" for more information.
>>> 

Interrupted

# URPC Server - Start from library

In [1]:
%connect esp32_cop

from urpc import async_serve
import uasyncio as asyncio

configuration = {
    # particle-argon with on-board esp32
    'argon':      { 'rx': 19, 'tx': 22, 'baudrate': 1000000 },
    'argon_flow': { 'rx': 19, 'tx': 22, 'rts': 0, 'cts': 26, 'baudrate': 1_000_000 },

    # huzzah32 as esp32 co-processor. tried on feather stm32f405.
    'huzzah32':   { 'rx': 16, 'tx': 17, 'baudrate': 1000000, 'rts': 18, 'cts': 19 },
}


def start_server(config, enable_net_server=False):

    uart = machine.UART(2, **config)
    uart.init(bits=8, stop=1, txbuf=1024, rxbuf=1024, timeout=20000)

    async def main(uart):
        print("starting urpc server ...")
        asyncio.create_task(async_serve(uart))
        
        if enable_net_server:
            import net_repl
            import secrets
            asyncio.create_task(net_repl.async_serve(
                port=54321, 
                password=getattr(secrets, 'net_pwd', None)))
            
        asyncio.get_event_loop().run_forever()

    asyncio.run(main(uart))

    
config = configuration['huzzah32']
start_server(config, enable_net_server=False)

Connected to esp32_cop @ serial:///dev/cu.usbserial-014352DD
with projects [secrets, airlift/libs/server]
starting urpc server ...


Interrupted

In [1]:
import gc
print(gc.mem_free())

Connected to esp32_cop @ serial:///dev/cu.usbserial-014352DD
with projects [secrets, airlift/libs/server]
99360
